In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Conv1D, Dropout, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.compat.v1.keras.backend as K
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import ModelCheckpoint
import random
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize


CNN 훈련

In [ ]:
# 레시피 데이터 불러오기
reci = pd.read_excel('./recipe_3.xlsx')
reci = reci[['레시피제목','요리명','요리재료내용','재료','조리순서','조회수','스크랩수','요리종류별명','재료_불용어 제외']]


# 레이블 인코딩
label_encoder = LabelEncoder()
reci['요리명_label'] = label_encoder.fit_transform(reci['요리명'])


# 클래스와 요리명을 딕셔너리로 매핑
class_to_recipe_name = dict(zip(reci['요리명_label'], reci['요리명']))



# 텍스트 데이터 전처리
X_text = reci['재료_불용어 제외']
y_label = reci['요리명_label']



# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_text, y_label, test_size=0.2, random_state=42)

# 토큰화 및 패딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)


# 패딩 길이 설정(34로 설정)
maxlen =34
X_train_padded = pad_sequences(sequences_train, maxlen=maxlen)
X_test_padded = pad_sequences(sequences_test, maxlen=maxlen)

# 토크나이저 저장
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)


# 모델 구성
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=maxlen))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 체크포인트 콜백 설정
checkpoint_path = "best_model.h5"
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# 모델 훈련
model.fit(X_train_padded, y_train, epochs=30, batch_size=32, validation_data=(X_test_padded, y_test),callbacks=[checkpoint_callback])

# 저장된 모델 불러오기
best_model = load_model(checkpoint_path)



Epoch 1/30


336/340 [============================>.] - ETA: 0s - loss: 3.4229 - accuracy: 0.2646
Epoch 1: val_accuracy improved from -inf to 0.59602, saving model to best_model.h5


e:\project3\myenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


340/340 [==============================] - 5s 14ms/step - loss: 3.4086 - accuracy: 0.2679 - val_loss: 1.8429 - val_accuracy: 0.5960
Epoch 2/30
334/340 [============================>.] - ETA: 0s - loss: 1.3584 - accuracy: 0.6835
Epoch 2: val_accuracy improved from 0.59602 to 0.67527, saving model to best_model.h5
340/340 [==============================] - 9s 27ms/step - loss: 1.3541 - accuracy: 0.6840 - val_loss: 1.4175 - val_accuracy: 0.6753
Epoch 3/30
339/340 [============================>.] - ETA: 0s - loss: 0.8450 - accuracy: 0.8032
Epoch 3: val_accuracy improved from 0.67527 to 0.69333, saving model to best_model.h5
340/340 [==============================] - 4s 12ms/step - loss: 0.8449 - accuracy: 0.8032 - val_loss: 1.2911 - val_accuracy: 0.6933
Epoch 4/30
339/340 [============================>.] - ETA: 0s - loss: 0.5931 - accuracy: 0.8614
Epoch 4: val_accuracy improved from 0.69333 to 0.71508, saving model to best_model.h5
340/340 [==============================] - 5s 15ms/step - 

In [ ]:
# # 전체 함수

# # 치환 함수 정의
# def normalize_word(word):
#     replacements = {
#         "달걀": "계란",
#         "오뎅": "어묵",
#         "무우": "무",
#         "대파": "파",
#         "씨레기": "시래기"
#         # 추가적인 치환 사항을 여기에 추가할 수 있습니다.
#     }
#     return replacements.get(word.strip(), word.strip())  # strip()을 사용하여 공백을 제거하고, get()을 사용하여 해당하는 값이 없을 경우 입력 값을 그대로 반환합니다.



# # 사용자의 입력을 받는 함수 정의
# def get_user_choice():
#     while True:
#         choice = input("쿠킹파트너 : 음식명 검색은 0, 재료명 검색은 1을 입력해 주세요~^^")
#         if choice in ['0','1']:
#             return choice
#         else:
#             print("쿠킹파트너: 올바른 선택지를 입력해주세요~!!")


# # 음식 카테고리 1 선택 함수 정의
# def show_categories(df):
#     categories = df['요리종류별명'].unique()
#     for idx, category in enumerate(categories, start=1):
#         print(f"{idx}.{category}")
#     return categories


# # 음식 카테고리 1 내 요리명 선택 함수 정의
# def show_recipes_by_category(df, category):
#     filterd_df = df[df['요리종류별명'] == category]
#     recipes = filterd_df['요리명'].unique()
#     for idx, recipe in enumerate(recipes, start=1):
#         print(f"{idx}.{recipe}")
#     return recipes


# # 요리명 선택 후 추천 요리 방법 함수 정의
# def recommend_recipes(df, recipe_name):
#     filtered_df = df[df['요리명'].str.contains(recipe_name)]
#     sorted_df = filtered_df.sort_values(by=['조회수', '스크랩수'], ascending=[False, False])
#     top_recipes = sorted_df.head(3)

#     responses = []
#     for i, match in enumerate(top_recipes.iterrows(), start=1):  # 최대 3개까지 출력
#         response = f"\n<{i}. {match[1]['레시피제목']}>\n\n재료:\n{match[1]['요리재료내용']}\n\n조리순서:\n{match[1]['조리순서']}"
#         if i == 1:
#             response = f"\n 쿠킹파트너 : {match[1]['요리명']} 레시피를 알려드리겠습니다.\n" + response
#         responses.append(response)

#     return '\n'.join(responses)


# # 모델 로드 함수 정의
# def load_trained_model(model_path):
#     return load_model(model_path)

# # Tokenizer 로드 및 텍스트 전처리 함수 정의
# def load_tokenizer(tokenizer_path):
#     with open(tokenizer_path, 'rb') as f:
#         tokenizer = pickle.load(f)
#     return tokenizer

# def prepare_text(text, tokenizer, maxlen=34):
#     sequences = tokenizer.texts_to_sequences([text])
#     padded_sequences = pad_sequences(sequences, maxlen=maxlen)
#     return padded_sequences





# #재료명을 기반으로 요리 레시피 추천 함수 정의
# def recommend_recipes_by_ingredients_ml(model, tokenizer, ingredients, data):

#     # 텍스트를 정렬된 상태로 합치기
#     text = ' '.join(sorted(ingredients))
#     X_text = prepare_text(text, tokenizer)


#     # 모델로 예측
#     predicted_class = model.predict(X_text).argmax(axis=1)[0]
#     predicted_recipe_name = class_to_recipe_name[predicted_class] # 인덱스를 요리명으로 변환

#     #예측된 클래스를 사용하여 해당 레시피를 추천
#     recommended_recipes = data[data['요리명'] == predicted_recipe_name]

#     #조회수 및 스크랩수로 정렬
#     recommended_recipes = recommended_recipes.sort_values(by=['조회수','스크랩수'], ascending =[False, False]).head(3)


#     responses =[]
#     if not recommended_recipes.empty:
#         for i , match in enumerate(recommended_recipes.iterrows(), start=1): #최대 3개까지 출력
#             response =  f"\n<{i}. {match[1]['레시피제목']}>\n\n재료:\n{match[1]['요리재료내용']}\n\n조리순서:\n{match[1]['조리순서']}"
#             if i == 1:
#                 response = f"\n 쿠킹파트너 : {match[1]['요리명']} 레시피를 알려드리겠습니다.\n" + response
#             responses.append(response)
#     else:
#         responses.append("쿠킹파트너: 해당하는 레시피가 없습니다.")

#     return '\n'.join(responses), recommended_recipes





# # 메인 실행 함수
# def main():
#     model = load_trained_model('best_model.h5')
#     tokenizer = load_tokenizer('tokenizer.pkl')

#     user_choice = get_user_choice()

#     if user_choice == '0':
#         categories = show_categories(reci)
#         while True:
#             category_choice = input("쿠킹파트너 : 1~10까지의 숫자를 입력하거나 카테고리명을 입력하세요: ")
#             if category_choice.isdigit():
#                 category_choice = int(category_choice)
#                 if 1 <= category_choice <= len(categories):
#                     selected_category = categories[category_choice -1]
#                     break

#             elif category_choice in categories:
#                 selected_category = category_choice
#                 break
#             print("쿠킹파트너 : 다시 선택해주세요~")

#         recipes = show_recipes_by_category(reci, selected_category)
#         while True:
#             recipe_choice = input( "쿠킹파트너: 숫자를 입력하거나 요리명을 입력하세요: ")
#             if recipe_choice.isdigit():
#                 recipe_choice = int(recipe_choice)
#                 if 1 <= recipe_choice <= len(recipes):
#                     selected_recipe = recipes[recipe_choice -1]
#                     break

#             elif recipe_choice in recipes:
#                 selected_recipe = recipe_choice
#                 break
#             print("쿠킹파트너: 다시 선택해주세요~")

#         print(recommend_recipes(reci, selected_recipe))


#     elif user_choice == '1':
#         ingredients = input("쿠킹파트너: 재료를 3가지 입력하세요 (쉼표로 구분해주세요^^): ").split(',')
#         ingredients =[normalize_word(ingredient.strip()) for ingredient in ingredients]
#         if len(ingredients) !=3:
#             print("쿠킹 파트너 : 재료를 정확히 3가지 입력해주세요~")
#             return


#         recommended_responses, recommended_data = recommend_recipes_by_ingredients_ml(model, tokenizer, ingredients, reci)
#         print(recommended_responses)

#     else:
#         print("쿠킹파트너 : 0과 1중에 입력해주세요")

# if __name__ == "__main__":
#     main()

1/1 [==============================] - 0s 286ms/step

 쿠킹파트너 : 김치전 레시피를 알려드리겠습니다.

<1. 초간단 백종원 김치전>

재료:
[재료] 김치| 부침가루 [양념] 고추가루 1큰술| 설탕(신김치일 경우) 1작은술

조리순서:
[조리순서]
1.김치를 가위로 잘게 썰어주세요. 부침가루를 넣어주세요. 김치와 부침가루는 2:1 비율로 넣어주세요.
2.물, 고추가루, 설탕을 넣어 섞어주세요. 고추가루는 색을 내기 위함이기도 해요.
3.달군 팬에 기름을 넉넉하게 두르고 반죽을 한숟가락씩 떠서 동그랗게 만든후에 센불에서 바삭하게 구워주면 완성입니다.

<2. 집밥 백선생  백종원 김치전 레시피>

재료:
[재료] 김치| 부침가루| 물| 식용류| 설탕| 참치

조리순서:
[조리순서]
1.김치를 갈기갈기 썰어주세요.
2.김치, 부침가루, 물로 계량하고 설탕과 참치를 넣으셨더라구요;;; 아무튼 설탕 투하.
3.참치캔 기름 빼고 투하.
4.김치:부침가루 = 2:1 로 투하.
5.됨직하게 찬물 살살 넣어가며 쉐이킹.
6.식용유를 붓고 부쳐주세요.

<3. 집밥 백선생 김치전 레시피>

재료:
[재료] 김치 2컵| 물 1컵| 부침가루1컵| 참치캔 1/2캔

조리순서:
[조리순서]
1.먼저 볼에 김치를 2컵 담았어요. 가위로 싹둑싹둑 잘라주세요. 도마에 자르면 도마에 김치물이 드니까 가위로 잘라주시면 된답니다.
2.부침가루도 넣어주시구요.
3.물도 넣어주었어요.
4.참치캔 반캔도 넣어 섞어주세요.
5.한 주걱 퍼서 달군팬에 기름 두르고 올려주시면 된답니다.
6.뒤집개로 밑 부분을 샤샤삭 넣고 확 뒤집으면 완성. 전도 너무 자주 뒤집으로 모양이 다 흐트러지니까 어느정도 익으면 뒤집어주세요.
